In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
transform1 = transforms.Compose([transforms.RandomGrayscale(p=0.4), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform1)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1024, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader= torch.utils.data.DataLoader(testset, batch_size=1024, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer','dog','frog','horse','ship','truck')

import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, 3)
        self.batchnorm1 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(128, 256, 3)
        self.batchnorm2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(256, 512, 3)
        self.batchnorm3 = nn.BatchNorm2d(512)
        self.fc1 = nn.Linear(512 * 2 * 2, 10)
        self.dropout = nn.Dropout2d(p=0.3)

    def forward(self,x):
        x = self.pool(self.batchnorm1(F.relu(self.conv1(x))))
        x = self.pool(self.batchnorm2(F.relu(self.conv2(x))))
        x = self.pool(self.batchnorm3(F.relu(self.conv3(x))))
        x = x.view(-1, 512 * 2 * 2)
        x = self.dropout(x)
        x = F.softmax(self.fc1(x))
        return x

net = Net()
# torch.cuda.set_device(1)
torch.cuda.set_device(0)
device = torch.device("cuda:0 " if torch.cuda.is_available() else "cpu")
print(device)

# if torch.cuda.device_count() > 1:
#   print("Let's use", torch.cuda.device_count(), "GPUs!")
#   # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#   net = nn.DataParallel(net)

net.to(device)

summary(net, (3, 32, 32))

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
# optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9,0.999), eps=1e-08, weight_decay=5e-4)
optimizer = optim.RMSprop(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

max_epoch = 50

for epoch in range(max_epoch):

    scheduler.step()
    running_loss = 0.0
    epoch_correct = 0
    epoch_total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data,1)
        epoch_total += labels.size(0)
        epoch_correct += (predicted == labels).sum().item()

        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # if i % 2000 == 1999:
        #     print ('[%d, %5d] loss: %3f' % (epoch+1, i+1, running_loss / 2000))
        #     running_loss = 0.0
    print('[epoch %d/%d] loss:%3f accuracy:%3f' % (epoch + 1, max_epoch, running_loss / i, 100 * epoch_correct / epoch_total))
print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy:%d %%' % (100 * correct / total))


Files already downloaded and verified
Files already downloaded and verified
cuda:0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 30, 30]           3,584
       BatchNorm2d-2          [-1, 128, 30, 30]             256
         MaxPool2d-3          [-1, 128, 15, 15]               0
            Conv2d-4          [-1, 256, 13, 13]         295,168
       BatchNorm2d-5          [-1, 256, 13, 13]             512
         MaxPool2d-6            [-1, 256, 6, 6]               0
            Conv2d-7            [-1, 512, 4, 4]       1,180,160
       BatchNorm2d-8            [-1, 512, 4, 4]           1,024
         MaxPool2d-9            [-1, 512, 2, 2]               0
        Dropout2d-10                 [-1, 2048]               0
           Linear-11                   [-1, 10]          20,490
Total params: 1,501,194
Trainable params: 1,501,194
Non-trainable params: 0
--------

/home/jonric/.virtualenvs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[epoch 1/50] loss:2.139719 accuracy:35.790000
[epoch 2/50] loss:1.994354 accuracy:50.330000
[epoch 3/50] loss:1.934658 accuracy:56.498000
[epoch 4/50] loss:1.900382 accuracy:60.010000
[epoch 5/50] loss:1.878708 accuracy:62.296000
[epoch 6/50] loss:1.862945 accuracy:63.950000
[epoch 7/50] loss:1.850894 accuracy:65.074000
[epoch 8/50] loss:1.840379 accuracy:66.240000
[epoch 9/50] loss:1.830735 accuracy:67.242000
[epoch 10/50] loss:1.829451 accuracy:67.280000
[epoch 11/50] loss:1.792970 accuracy:70.848000
[epoch 12/50] loss:1.770053 accuracy:73.156000
[epoch 13/50] loss:1.760066 accuracy:74.242000
[epoch 14/50] loss:1.751322 accuracy:75.220000
[epoch 15/50] loss:1.742159 accuracy:76.262000
[epoch 16/50] loss:1.734872 accuracy:76.944000
[epoch 17/50] loss:1.730021 accuracy:77.506000
[epoch 18/50] loss:1.724559 accuracy:77.986000
[epoch 19/50] loss:1.717637 accuracy:78.678000
[epoch 20/50] loss:1.714535 accuracy:79.076000
[epoch 21/50] loss:1.706113 accuracy:79.932000
[epoch 22/50] loss:1.7